# NEWS 파일(json)을 Bigquery에 저장

#### 이슈

* 이 노트북에서는 이미 다운로드 받은 NEWS 파일을 전송하는 것이 목적이나, 향후에는 newscrawler와 결합해야 한다. 
* 이때 newscrawler이 수집한 url 중 이미 다운로드한 url이 있는지를 검사(url selecting)해야 한다. 
* 이를 Bigquery에서 직접 참조하여 검사하려면, Bigquery에 저장된 url list를 다운로드 해야하는데
* **쿼리비용**과 **다운로드 파일용량**이 걱정된다. 
* 예상 쿼리량: 0.4MB / 뉴스 1만개당 (평일 하루평균 다운로드 갯수 약 8-9천개), 즉 한달 지나면 약 10MB 증가
* 예상 파일용량: 0.5MB / 뉴스 1만개당

In [4]:
11/221574 * 10000

0.4964481392221109

## 아이디어

* newsids.pickle 파일에 url 집합(set)을 통째로 저장한다
* 뉴스를 새로 다운로드할 때마다, newsids의 url 갯수와 Bigquery상의 레코드갯수(meta)를 단순비교한다

In [1]:
import os
import sys
import json
import requests
import pickle
import gzip
import pandas as pd
import pandas_gbq as gbq
from pathlib import Path
from google.oauth2 import service_account
from google.cloud import bigquery
from IPython.core.debugger import set_trace
from tqdm.auto import tqdm
tqdm.pandas()

## Configuration

In [2]:
proj = 'global-news-crawl'
table_downloaded = 'news_dataset.downloaded'
table_trashed = 'news_dataset.trashed'
credentials = service_account.Credentials.from_service_account_file('global-news-crawl-c48d7cd9aa81.json')
newsids_pkl = 'newsids.pickle'

## NewsIDs helper class

In [3]:
class NewsIDs:
    def __init__(self, fpath, init=False):
        self.fpath = fpath
        
        if init:
            self.ids = set()
                
        else:
            with open(fpath, 'rb') as f:
                self.ids = pickle.load(f)
        
    def has(self, id):
        return id in self.ids
    
    @property
    def size(self):
        return len(self.ids)
    
    def push(self, *ids_to_add):
        self.ids |= set(ids_to_add)
        
    def save(self):
        with open(self.fpath, 'wb') as f:
            pickle.dump(self.ids, f)
            
    def push_and_save(self, *ids_to_add):
        self.push(*ids_to_add)
        self.save()

## newsids에 저장된 url 갯수와 Bigquery 레코드갯수가 같은지 확인

In [4]:
newsids = NewsIDs(newsids_pkl, init=False); newsids.size

352447

In [5]:
bq_client = bigquery.Client(project=proj, credentials=credentials)
_table_downloaded = bq_client.get_table(table_downloaded)
_table_trashed = bq_client.get_table(table_trashed)

assert (_table_downloaded.num_rows + _table_trashed.num_rows) == newsids.size, 'size of newsids mismatched'

## 파일에서 뉴스정보 추출
나중에는 필요없는 과정이다. 다운로드 받은 즉시 바로 Bigquery에 전송할 것이므로

In [6]:
def extract_contents(newsids=None, where=None, n=1000):
    df = {}
    _n = 0
    
    for file in Path(where).glob('**/*.json'):
        id = file.stem
        
        if not newsids.has(id):
            try:
                js = json.loads(file.read_text())
            
                if 'authors' in js:
                    js['authors'] = ', '.join(js['authors'])

                df[id] = js

                _n += 1
                print('\r{}'.format(_n), end='')
                if _n == n: break
                    
            except:
                print(file)
           
    df = pd.DataFrame.from_dict(df, orient='index')
    df.index.name = 'id'
    return df.reset_index()

In [7]:
df_downloaded = extract_contents(newsids=newsids, where='newsdata/downloaded', n=50000); print('\n')
df_trashed = extract_contents(newsids=newsids, where='newsdata/trashed', n=30000)

3772

2280

## downloaded와 trashed 간에 겹치는 게 없는지 확인
기존의 뉴스파일을 Bigquery에 전송하는 과정에서, 이 둘간에 겹치는 사례가 종종 있었다
(2019.10.18)

In [8]:
intersect = set(df_trashed.id) & set(df_downloaded.id)
assert len(intersect) == 0, 'ids overlapping'

## downloaded, trashed 전체 url을 newsids에 저장

In [9]:
newsids.push_and_save(*df_downloaded.id, *df_trashed.id); newsids.size

358499

## Bigquery에 업로드

In [10]:
gbq.to_gbq(df_downloaded, table_downloaded, project_id=proj, if_exists='append', chunksize=1000, credentials=credentials)
gbq.to_gbq(df_trashed, table_trashed, project_id=proj, if_exists='append', chunksize=1000, credentials=credentials)

4it [00:35,  8.01s/it]
3it [00:13,  3.47s/it]


# 이 노트북 방식은 drop한다 (2019.10.19)

* newsids.pickle 아이디어는 썩 괜찮았으나, Bigquery상에 저장된 실질적인 url을 확인하지 않는 다는 결점이 있다
* 특히 Bigquery에 업로드하는 과정에서 HTTP연결오류 등이 발생하면 더욱 난감해진다
* 쿼리비용 등이 걱정되지만, 역시 Bigquery에서 직접 url list를 가져오는 것이 가장 확실한 방법인것 같다
* 비용, 다운용량 등은 나중에 다른 아이디어가 생기겠지....